## Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df=application_df.drop(['EIN','NAME'],axis=1)

In [40]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [41]:
for col in application_df.columns:
    print(f"{col} {application_df[col].nunique()}")

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [42]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [43]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe

application_types_to_replace=['T9','T13','T12','T2','T14','T25','T15','T29','T17']

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [44]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C5200        1
C1283        1
C3700        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [58]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
application_df=application_df.groupby('CLASSIFICATION').filter(lambda x: len(x) > 1)
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [59]:
classification_dict=application_df['CLASSIFICATION'].value_counts().to_dict()

classifications_to_replace=[]

for key, value in classification_dict.items():
    if value <1000:
        classifications_to_replace.append(key)
       

In [60]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [48]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [49]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df_encoded =pd.get_dummies(application_df, drop_first=True)
application_df_encoded.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7',
       'APPLICATION_TYPE_T8', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000',
       'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000', 'CLASSIFICATION_Other',
       'USE_CASE_Heathcare', 'USE_CASE_Other', 'USE_CASE_Preservation',
       'USE_CASE_ProductDev', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_1-9999',
       'INCOME_AMT_10000-24999', 'INCOME_AMT_100000-499999',
       'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 'INCOME_AMT_25000-99999',
       'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_Y'],
      dtype='object')

In [50]:
application_df_encoded['IS_SUCCESSFUL'].value_counts()

1    18243
0    16030
Name: IS_SUCCESSFUL, dtype: int64

In [51]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X=application_df_encoded.drop(['IS_SUCCESSFUL'],axis=1)
y=application_df_encoded['IS_SUCCESSFUL']

In [52]:
X.shape

(34273, 36)

In [53]:
y.shape

(34273,)

In [54]:
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=36,activation='relu',input_dim=36))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=18,activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 36)                1332      
_________________________________________________________________
dense_1 (Dense)              (None, 18)                666       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 19        
Total params: 2,017
Trainable params: 2,017
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
#  YOUR CODE GOES HERE
#save_best_only=True,
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
#  YOUR CODE GOES HERE
from keras.callbacks import ModelCheckpoint

filepath="Weights/weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='accuracy',
                             verbose=1, 
                             mode='max',
                            period=5)

fit_model=nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7390
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7394
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7376: 0s - loss: 0.5350 - accuracy
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7384
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7386

Epoch 00005: saving model to Weights/weights-improvement-05-0.74.hdf5
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7386: 0s - loss: 0.5
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7391
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7390
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 

In [130]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7377
Loss: 0.5606107115745544, Accuracy: 0.7376589775085449


In [36]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
from tensorflow import keras
nn.save('model_1.h5')
nn_model = keras.models.load_model('model_1.h5')